In [1]:
import os
import sys
import pandas as pd
sys.path.append("../..")
import goatwu.basicfunc as basefunc
import goatwu.imgclassifier.dataset.mixing as mixing
import goatwu.imgclassifier.dataset.loader as loader
import goatwu.imgclassifier.dataset.trans as trans
import goatwu.imgclassifier.function as func


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data

import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

In [2]:
img_path = "../../datasets/classify-leaves/"
train_csv_path = "../../datasets/classify-leaves/train.csv"
test_csv_path = "../../datasets/classify-leaves/test.csv"

In [3]:
df = pd.read_csv(train_csv_path)
df.head(5)

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [4]:
leaves_labels = sorted(list(set(df['label'])))
n_classes = len(leaves_labels)
print(n_classes)

176


In [5]:
cls2num = dict(zip(leaves_labels, range(n_classes)))
num2cls = {v : k for k, v in cls2num.items()}

In [6]:
def mytransform():
    train_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    valid_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    return train_transform, valid_transform


train_transform, valid_transform = mytransform()
trainset = mixing.TrainDataset(csv_path=train_csv_path, 
                               img_path=img_path, cls2num=cls2num, 
                               transform=train_transform)

# 训练

## ResNext50

In [7]:
import torchvision.models as models
resnext = models.resnext50_32x4d(pretrained=True, progress=True)
resnext.fc = nn.Linear(resnext.fc.in_features, 176)

In [8]:
resnext.__class__.__name__ = 'resnext50_32x4d'
print(resnext)

resnext50_32x4d(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kerne

In [9]:
device = basefunc.try_gpu()
num_epochs, batch_size = 30, 32

optimizer = func.adamw_default(net=resnext)
train_loader_func = loader.generate_cutmix_loader_func(num_class=176)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
loss = CutMixCrossEntropyLoss(True)

# optimizer = func.adam_train_fine_tuning(net=resnext, lr=1e-2)
# loss = nn.CrossEntropyLoss()
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, 
#                                                        verbose=True, min_lr=0.0000001)

init_func = func.pretrained_init
kfold = KFold(n_splits=5, shuffle=True)


In [ ]:
func.train(resnext, trainset, None, num_epochs, batch_size, optimizer, loss, 
           use_tensorboard=True, init_func=init_func, 
           train_loader_func=train_loader_func, use_cutmix=True,
           device=device, scheduler=scheduler, kfold=kfold)

## Efficientnet-B2

In [10]:
import torchvision.models as models
efficientnet = models.efficientnet_b2(pretrained=True, progress=True)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, 176)
efficientnet.__class__.__name__ = 'efficientnet_b2'
print(efficientnet)

efficientnet_b2(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation

In [11]:
device = basefunc.try_gpu()
num_epochs, batch_size = 50, 32

optimizer = func.adamw_default(net=efficientnet, lr=1e-4)
train_loader_func = loader.generate_cutmix_loader_func(num_class=176)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
loss = CutMixCrossEntropyLoss(True)

def init_efficientnet(net):
    nn.init.xavier_uniform_(net.classifier[1].weight);

init_func = init_efficientnet
kfold = KFold(n_splits=5, shuffle=True)


In [ ]:
func.train(efficientnet, trainset, None, num_epochs, batch_size, optimizer, loss, 
           use_tensorboard=True, init_func=init_func, 
           train_loader_func=train_loader_func, use_cutmix=True,
           device=device, scheduler=scheduler, kfold=kfold)

## DenseNet-161

In [7]:
import torchvision.models as models
densenet = models.densenet161(pretrained=True, progress=True)
densenet.classifier = nn.Linear(densenet.classifier.in_features, 176, bias=True)
densenet.__class__.__name__ = 'densenet-161'
print(densenet)

densenet-161(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [8]:
device = basefunc.try_gpu()
num_epochs, batch_size = 50, 16

optimizer = func.adamw_default(net=densenet, lr=1e-4)
train_loader_func = loader.generate_cutmix_loader_func(num_class=176)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
loss = CutMixCrossEntropyLoss(True)

def init_densenet(net):
    nn.init.xavier_uniform_(net.classifier.weight);

init_func = init_densenet
kfold = KFold(n_splits=5, shuffle=True)


In [ ]:
func.train(densenet, trainset, None, num_epochs, batch_size, optimizer, loss, 
           use_tensorboard=True, init_func=init_func, 
           train_loader_func=train_loader_func, use_cutmix=True,
           device=device, scheduler=scheduler, kfold=kfold)

# 测试和提交

In [8]:
testset = mixing.TestDataset(csv_path=test_csv_path, 
                             img_path=img_path, num2cls=num2cls,
                             transform=valid_transform)

## ResNext50

In [8]:
import torchvision.models as models
resnext = models.resnext50_32x4d(pretrained=True, progress=True)
resnext.fc = nn.Linear(resnext.fc.in_features, 176)
resnext.__class__.__name__ = 'resnext50_32x4d'

func.predict(resnext, testset, batch_size=128, data_path=test_csv_path, 
             device=basefunc.try_gpu(), kfold=5, use_tta=True, tta_height=200)

100%|███████████████████████████████████████████████████████████████████████████████████| 69/69 [01:02<00:00,  1.10it/s]


In [7]:
submission_dir = './submission/resnext50_32x4d/'
kfold = 5
func.voting_kfold(submission_dir, kfold, cls2num, num2cls, name='resnext50_32x4d.csv')

## Efficientnet-B2

In [8]:
import torchvision.models as models
efficientnet = models.efficientnet_b2(pretrained=True, progress=True)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, 176)
efficientnet.__class__.__name__ = 'efficientnet_b2'

func.predict(efficientnet, testset, batch_size=128, data_path=test_csv_path, 
             device=basefunc.try_gpu(), kfold=5, use_tta=True, tta_height=200)

100%|███████████████████████████████████████████████████████████████████████████████████| 69/69 [00:36<00:00,  1.91it/s]


In [8]:
submission_dir = './submission/efficientnet_b2/'
kfold = 5
func.voting_kfold(submission_dir, kfold, cls2num, num2cls, name='efficientnet_b2.csv')

## DenseNet-161

In [9]:
import torchvision.models as models
densenet = models.densenet161(pretrained=True, progress=True)
densenet.classifier = nn.Linear(densenet.classifier.in_features, 176, bias=True)
densenet.__class__.__name__ = 'densenet-161'

func.predict(densenet, testset, batch_size=128, data_path=test_csv_path, 
             device=basefunc.try_gpu(), kfold=5, use_tta=True, tta_height=200)

100%|███████████████████████████████████████████████████████████████████████████████████| 69/69 [01:54<00:00,  1.66s/it]


In [9]:
submission_dir = './submission/densenet-161/'
kfold = 5
func.voting_kfold(submission_dir, kfold, cls2num, num2cls, name='densenet-161.csv')

# voting all models

In [8]:
submission_dir = './submission/submission.csv'
resnext_dir = './submission/resnext50_32x4d/resnext50_32x4d.csv'
efficientnet_dir = './submission/efficientnet_b2/efficientnet_b2.csv'
densenet_dir = './submission/densenet-161/densenet-161.csv'

func.voting_models(cls2num, num2cls, submission_dir, resnext_dir, efficientnet_dir, densenet_dir)
